In [1]:
from IPython import get_ipython
ip = get_ipython()
ip.run_line_magic("reload_ext", "autoreload")  # these will enable module autoreloading
ip.run_line_magic("autoreload", "2")

### Get List of Reports Available

In [2]:
import os
from glob import glob
import pandas as pd
from ExportProvider.IBRK.ExportReports.CashTransactions import CashTransactionReport
from ExportProvider.IBRK.ExportReports.CashTransactions import CashTransactionType
from ReportingStrategies.Slovenia.DividendsReport import DividendsReport
import ExportProvider.IBRK.Mappers.CashTransactions as ibrk

rootOfProject = os.getcwd()

reportsDirectory = os.path.join(rootOfProject, "BrokerExports")
generatedDirectory = os.path.join(rootOfProject, "GeneratedReports")

cashReportsfiles = glob("CashTransactions_*.csv", root_dir=reportsDirectory)
print(cashReportsfiles)


def loadFileAndConvertToReport(file: str):
    fileLocation = os.path.join(reportsDirectory, file)
    loaded = pd.read_csv(fileLocation)
    loaded = loaded.rename(columns={"Put/Call":"PutSlashCall","Date/Time":"DateAndTime"})

    # find DateTime in first column, as that marks CurrencyConversion start
    dateTimeIndex = loaded[loaded['ClientAccountID'] == 'Date/Time'].index[0]

    goodRows = loaded[:dateTimeIndex].itertuples(name="Row", index=False)

    report = CashTransactionReport(goodRows)
    return report

cashReports = list(map(loadFileAndConvertToReport, cashReportsfiles))

mergedReports = CashTransactionReport.mergeCashTransactionReports(cashReports)


dividends = mergedReports.getTransactionRowsOfType(CashTransactionType.Dividends)
taxes = mergedReports.getTransactionRowsOfType(CashTransactionType.WITHOLDING_TAX)

dividends = list(map(ibrk.convertDividendCashTransactionRowToDividend, dividends))
taxes = list(map(ibrk.convertWitheldTaxCashTransactionRowToWitheldTax, taxes))

dividendReport = DividendsReport(dividends, taxes)

reported = dividendReport.getDividentReportDataFrame()

reported.to_csv(os.path.join(generatedDirectory, 'export.csv'), index=False)
print(reported)


['CashTransactions_2022.csv', 'CashTransactions_2023.csv']
Failed for ISIN: US26152H3012
'ZA'


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=&crumb=GLs57BTI1Js


Failed for ISIN: US9663875089
'timeseries'
Failed for ISIN: US26152H3012
'ZA'
Failed for ISIN: US9663875089

   Datum prejema dividend Davčna številka izplačevalca dividend  \
0           01. 04. 2022.                                         
1           27. 04. 2022.                                         
2           16. 05. 2022.                                         
3           01. 07. 2022.                                         
4           03. 10. 2022.                                         
..                    ...                                   ...   
85          28. 11. 2023.                                         
86          28. 11. 2023.                                         
87          30. 11. 2023.                                         
88          11. 12. 2023.                                         
89          21. 12. 2023.                                         

   Identifikacijska številka izplačevalca dividend  \
0                               